__Steady state analysis of AC circuit__  
---   
The project for *fundumentals of electrical engnieering* course  
***
**M. Sadra Mahmoudi**  
Winter 2023



In [1]:
#Importing necessary libraries
from tabulate import tabulate as tbl
import numpy as np

## Define the Element Classes

### Class for R, L, C , Z

In [2]:
class CircuitElement:
  """
  Represents a circuit element in an AC circuit.
  """
  def __init__(self, node1, node2, element_type, value):
    """
    Initialize a CircuitElement object.

    Parameters:
    - node1: the index of the first node connected to the element
    - node2: the index of the second node connected to the element
    - element_type: the type of the element (R, Z, C, L)
    - value: the value of the element
    """
    self.node1 = node1
    self.node2 = node2
    self.element_type = element_type
    self.value = value

  def impedance(self, frequency):
    """
    Calculate the complex impedance of the element at the given frequency.

    Parameters:
    - frequency: the frequency in Hz

    Returns:
    - the complex impedance of the element
    """
    if self.element_type == 'R':
      return self.value
    
    elif self.element_type == 'L':
      return 1j*2*np.pi*frequency*self.value
    
    elif self.element_type == 'C':
      return 1/(1j*2*np.pi*frequency*self.value)
    
    elif self.element_type == 'Z':
      return self.value
    
    else:
      raise ValueError(f"Invalid element type: {self.element_type}")


In [3]:
sample = CircuitElement(node1 = 1,
                   node2 = 2,
                   element_type= 'C',
                   value= .0005)

print(np.round(sample.impedance(frequency=50),2))

-6.37j


### Class for Sources

In [4]:
class Source:
  """
  Represents an AC voltage/current source in an AC circuit.
  """
  def __init__(self, node1, node2, element_type, value, phase):
    """
    Initialize a source object.

    Parameters:
    - node1: the index of the first node connected to the element
    - node2: the index of the second node connected to the element
    - value: the rms value
    - phase: the phase difference between the voltage and current in degrees

    """
    assert element_type in ['I', 'V'], "Invalid element type"

    
    self.node1 = node1
    self.node2 = node2
    self.element_type = element_type
    self.value = value
    self.phase = phase
    
  def complex_value(self):
    """
    Calculate the complex current/voltage of the AC source.
    """
    return self.value*np.exp(1j*np.deg2rad(self.phase))

In [5]:
sample = Source(node1= 4,
           node2 = 5,
           element_type= 'I',
           value= 10,
           phase= 20)

sample.complex_value().round(3)

(9.397+3.42j)

## Parse the Input

In [6]:
def parse_input(input: str):
      '''
      Parse the input text (with multiple lines)
      
      Returns:
      - A list containing circuit elements 
      - The frequency
       
      '''
      input_rows = [s.split(" ") for s in input.split('\n')]
      
      for i in range(len(input_rows)):
            if input_rows[i][0] == 'f':
                  f = input_rows.pop(i)
                  
      frequency = float(f[-1])
      
      def parse_element(element_list):
            """
            Parse a list representing a circuit element/source into a proper object.

            Parameters:
            - element_list: list of values representing the element
            (for example: [1, 2, 'R', 3])

            Returns:
            - a CircuitElement or Source object
            """
            
            node1 = int(element_list[0])
            node2 = int(element_list[1])
            element_type = element_list[2]
            if element_type != 'Z':
                  value = float(element_list[3])
            else:
                  value = complex(element_list[3])
            
            if element_type in ['R', 'C', 'L', 'Z']:
                  return CircuitElement(node1, node2, element_type, value)

            if element_type in ['V', 'I']:
                  phase = float(element_list[4])
                  return Source(node1, node2, element_type, value, phase)
            
            else:
                  raise ValueError(f"Invalid element type: {element_type}")
      
      elements = [parse_element(row) for row in input_rows]
      return elements, frequency


## Auxiliary Functions

In [7]:
def find_num_nodes(elements):
      counter = 1
      for i in elements:
            if i.node1 > counter:
                  counter = i.node1
            if i.node2 > counter:
                  counter = i.node2
      return counter

In [8]:
def split_elements(elements):
      """
      Divide the element into three categories: impedences, current sources, and voltage sources
      
      Parameters:
      - element: list of element

      Returns:
      - Three lists containing the element: impedences, current_sources, voltage_sources
      """
      impedences = []
      current_sources = []
      voltage_sources = []
      for i in elements:
            if i.element_type in ['R', 'C', 'L', 'Z']:
                  impedences.append(i)
            elif i.element_type == 'V':
                  voltage_sources.append(i)
            elif i.element_type == 'I':
                  current_sources.append(i)
      return impedences, current_sources, voltage_sources

In [9]:
def r2p(r: complex):
      '''transform a complex number from rectangular form to polar form'''
      angle = np.angle(r, deg = True)
      amp = np.abs(r)
      return str(np.round(amp,3)) + " < " + str(np.round(angle,2))

In [10]:
r2p(np.complex(1,4))

'4.123 < 75.96'

In [11]:
class supernodes:
      
      def __init__(self):
            from collections import defaultdict
            self.all_nodes = set() #set containing all the nodes that are recorded
            self.clusters = [] #list containing sets which represent supernodes
            self.connections = defaultdict(int) # stores the number of other nodes in a supernode is each node connected
      
      def add(self, node1, node2):
            self.connections[node1] += 1
            self.connections[node2] += 1
            
            if (node1 in self.all_nodes) and (node2 in self.all_nodes): #when two previously seperated supernodes are connected
                  for i in self.clusters:
                        if (node1 in i) and (node2 not in i):
                              temp1 = i.copy()
                              
                        elif (node2 in i) and (node1 not in i):
                              temp2 = i.copy()
                              
                  try:
                        self.clusters.append(temp1.union(temp2))
                        self.clusters.remove(temp1)
                        self.clusters.remove(temp2)
                  except:
                        pass
                  
            if node1 in self.all_nodes or node2 in self.all_nodes:
                  if node1 in self.all_nodes:
                        for i in self.clusters:
                              if node1 in i:
                                    i.add(node2)
                                    self.all_nodes.add(node2)
                                    break
                  if node2 in self.all_nodes:
                        for i in self.clusters:
                              if node2 in i:
                                    i.add(node1)
                                    self.all_nodes.add(node1)
                                    break         

                  
            else:
                  self.clusters.append(set([node1,node2]))
                  self.all_nodes.add(node1)
                  self.all_nodes.add(node2)
                  
      def connected_nodes(self, node, inclusive = False):
            '''returns a list containing nodes that are connected to a given node in a supernode'''
            assert node in self.all_nodes, 'invalid node'
            
            for i in self.clusters:
                  if node in i:
                        if inclusive:
                              return i
                        else:
                              return [j for j in i if j != node]

In [12]:
x = supernodes()
x.add(1,2)
x.add(3,4)
x.add(4,5)
x.add(2,3)

x.clusters

[{1, 2, 3, 4, 5}]

## Solver Function

In [13]:
def Solve(elements, frequency):

  # Initial Definitions
  num_nodes = find_num_nodes(elements)
  impedences, current_sources, voltage_sources = split_elements(elements)
  
  # Create the blank matrix for admittance matrix
  Y = np.zeros((num_nodes, num_nodes), dtype = complex) 
  
  # First, we add the impedences
  for z in impedences:
    Y[z.node1-1, z.node1-1] += 1/z.impedance(frequency)
    Y[z.node2-1, z.node2-1] += 1/z.impedance(frequency)
    Y[z.node1-1, z.node2-1] -= 1/z.impedance(frequency)
    Y[z.node2-1, z.node1-1] -= 1/z.impedance(frequency)
    
  
  # Then, current sources
  S = np.zeros(num_nodes, dtype = complex)
  for c in current_sources:
    S[c.node1-1] -= c.complex_value()
    S[c.node2-1] += c.complex_value()
  
  #Then, voltage sources
  su = supernodes()
  
  for v in voltage_sources:
    su.add(v.node1,v.node2)
  
  for v in voltage_sources:
    t1 = Y[v.node1-1].copy() #temporary variable to store the sum of selected rows (nodes in a supernode)
    t2 = S[v.node1-1].copy()
    
    for n in su.connected_nodes(v.node1): #finding all the supernodes
      t1 += Y[n-1]
      t2 += S[n-1]
      
    for i in su.connected_nodes(v.node1, inclusive= True): #replacing each row with sum of the connected rows
      Y[i-1] = t1
      S[i-1] = t2
      
    
    #an we should add the equation: v.node2 - v.node1 = v.value
    y = np.zeros((1,num_nodes), dtype = complex)
    y[0][v.node2-1] = 1
    y[0][v.node1-1] = -1
    Y = np.append(Y,y, axis=0)
    
    S = np.append(S, np.array([v.complex_value()]))
    
    
  #Setting V1 = 0 as the reference point
  reference_point = np.zeros((1,num_nodes), dtype = complex)
  reference_point[0][0] = 1
  
  Y = np.append(Y, reference_point, axis= 0)
  S = np.append(S, np.array([[0]], dtype= complex))
  
  # Solve for the phasor voltages
  V = np.linalg.lstsq(Y, S, rcond = None)[0]
  
  V = np.array(list(map(complex, V)))
  V = np.round(V, 2)
  
  Vpolar = np.array(list(map(r2p, V))).reshape(-1,1)
  V_names = np.array(["V" + str(i) for i in range(1,num_nodes + 1)]).reshape(-1,1)
  
  voltage_table = np.append(V_names, Vpolar, axis=1)
  
  print(tbl(voltage_table, headers = ['', 'A(rms) < phi'], tablefmt="github"))
  
  
  #Now we calculate the currents
  C = np.zeros((num_nodes, num_nodes), dtype = complex) #Currents: from point i to point j

  
  for z in impedences:
      From = z.node1-1
      To = z.node2-1
      i = (V[From] - V[To])/z.impedance(frequency)
      
      C[From, To] += i
      C[To, From] -= i
      
  for c in current_sources:
      From = c.node1-1
      To = c.node2-1
      i = c.complex_value() 
      
      C[From, To] += i
      C[To, From] -= i
      
  #Now we want to find the current passing through each voltage source — this was indeed the most tricky part!
  #First, the matrix on the left hand side of the equation
  m = np.zeros((num_nodes, len(voltage_sources))) 
  for i in range(len(voltage_sources)):
        v = voltage_sources[i]
        m[v.node1-1, i] -= 1
        m[v.node2-1, i] += 1
        
  #Then, right-hand side of the equation
  l = []
  for i in range(num_nodes):
    l.append((C[i,:].sum() - C[:,i].sum())/2)
  l = np.array(l, dtype= complex)
  
  #Now we can solve the equation to find the currents passing through voltage sources
  i_voltage_sources = np.linalg.lstsq(m, l, rcond = None)[0]

  for n in range(len(voltage_sources)):  # Then we add these to the C matrix which determines each current in the system
        v = voltage_sources[n]
        From = v.node1-1
        To = v.node2-1
        i = i_voltage_sources[n]
        C[From, To] += i
        C[To, From] -= i
        
        
  r2p_v = np.vectorize(r2p)
  Cpolar = r2p_v(C)
  
  t = np.array(['From', 'To', 'V', 'I', 'P', 'Q', 'S']).reshape(1,-1)
  
  for i in range(num_nodes):
    for j in range(num_nodes):
      if j>=i:
        c = C[i,j]
        if c != 0:
          v =  V[i] -V[j]
          s = np.conjugate(c)*v
          row = np.array([i+1, j+1, r2p(v), r2p(c), round(s.real,4), round(s.imag,4), r2p(s)]).reshape(1,-1)
          t = np.append(t, row, axis = 0)
        
  print('\n')  
  print( tbl(t, tablefmt="github" , headers = 'firstrow', stralign= 'center', numalign= 'center'))

  return voltage_table, Cpolar


## Running

In [18]:
input = '''1 2 V 1 90
1 4 Z -0.5j
1 3 I 2 20
3 4 I 1 90
2 3 Z 1+0.5j
f 1'''


In [19]:
elements, frequency = parse_input(input)
V, C = Solve(elements, frequency) 

|    | A(rms) < phi   |
|----|----------------|
| V1 | 0.0 < 0.0      |
| V2 | 1.0 < 90.0     |
| V3 | 2.605 < 38.45  |
| V4 | 0.5 < 0.0      |


|  From  |  To  |        V        |       I        |    P    |    Q    |       S        |
|--------|------|-----------------|----------------|---------|---------|----------------|
|   1    |  2   |   1.0 < -90.0   | 1.906 < 170.4  | -0.318  | 1.8797  |  1.906 < 99.6  |
|   1    |  3   | 2.605 < -141.55 |   2.0 < 20.0   | -4.9421 | -1.6492 | 5.21 < -161.55 |
|   1    |  4   |   0.5 < 180.0   |  1.0 < -90.0   |   -0    |  -0.5   |  0.5 < -90.0   |
|   2    |  3   | 2.132 < -163.09 | 1.907 < 170.34 | 3.6368  | 1.8184  | 4.066 < 26.57  |
|   3    |  4   |  2.235 < 46.45  |   1.0 < 90.0   |  1.62   |  -1.54  | 2.235 < -43.55 |
